In [1]:
import sys 
import joblib
sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/python_packages/WoF_post')
sys.path.append('/home/samuel.varga/python_packages/scikit-explain')
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/experiments')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')

from ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset, random_subsampler)
from os.path import join, exists
import sys
import argparse
#Appendages
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/')
sys.path.append('/home/samuel.varga/python_packages/ml_workflow/')
sys.path.append('/home/samuel.varga/python_packages/VargaPy/')
# Import packages 
import pandas as pd
import numpy as np
import sklearn
from os.path import join
from sklearn.linear_model import LogisticRegression
from hyperopt import hp
from main.io import load_ml_data
from ml_workflow.calibrated_pipeline_hyperopt_cv import norm_aupdc_scorer, norm_csi_scorer
from ml_workflow.tuned_estimator import TunedEstimator, dates_to_groups
from VargaPy.MlUtils import All_Severe, Drop_Unwanted_Variables, Train_Ml_Parser
from sklearn.model_selection import GroupKFold

lookup_file: /home/samuel.varga/python_packages/WoF_post/wofs/data/psadilookup.dat


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Load data

In [2]:
FRAMEWORK='POTVIN'
TIMESCALE='2to6'
target_scale='36'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = All_Severe(base_path, mode='train',
                                  target_scale=target_scale,
                                  FRAMEWORK=FRAMEWORK,
                                  TIMESCALE=TIMESCALE, Big=False)
X, ts_suff = Drop_Unwanted_Variables(X, original=False, training_scale=False, intrastormOnly=False, envOnly=False)

37752
63915
66734
Using new variables- dropping old 90th percentile
(1274972, 174)
all


## Plot Coefficient Ranks

In [3]:
from skexplain.common.importance_utils import to_skexplain_importance
import skexplain

base_path=f'/work/samuel.varga/projects/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
model_ind=2
model=joblib.load(join(base_path, f'mlModels/{target_scale}km/Varga_all_logistic_all_{target_scale}km_control_{model_ind}.joblib'))

# X and y are the training dataset
explainer = skexplain.ExplainToolkit(X=X,y=y)
coefs = model['model'].base_estimator.named_steps['model'].coef_[0]
coef_rank = to_skexplain_importance(coefs,
                                       estimator_name=name, 
                                       feature_names=X.columns, 
                                        method = 'coefs')

panels = [('Coefs.', 'LogisticRegression')]
fig = explainer.plot_importance(data=coef_rank, panels=panels, 
                                num_vars_to_plot=10, 
                               )

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [5]:
#model['model'].base_estimator.named_steps['model'].coef_
print(model['model'].base_estimator.named_steps['model'])
print(dir(model['model'].base_estimator.named_steps['model']))
#It should have coefficients after .fit is called during training -- is it saving the untrained model??

LogisticRegression(l1_ratio=0.001, max_iter=300, penalty='elasticnet',
                   random_state=42, solver='saga')
['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_feature_names', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol'